In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import matplotlib.pyplot as plt
from CLEANING_FUNCTIONS import *
from AQI_FUNCTIONS import *
from warnings import filterwarnings
filterwarnings('ignore')


In [80]:
tema= pd.read_csv('stacked_tema.csv')
tema

,UTCDateTime,mac_address,firmware_ver,hardware,current_temp_f,current_humidity,current_dewpoint_f,pressure,adc,mem,...,pm10_0_atm_b,pm2.5_aqi_cf_1_b,pm2.5_aqi_atm_b,p_0_3_um_b,p_0_5_um_b,p_1_0_um_b,p_2_5_um_b,p_5_0_um_b,p_10_0_um_b,gas
0,2024/03/05T20:29:46z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,72,19,28,863.65,NaN,36816,...,0.00,0,0,71.61,21.24,3.67,0.35,0.00,0.0,NaN
1,2024/04/12T11:31:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,83,53,64,1001.48,NaN,38496,...,21.14,68,68,2207.44,686.84,117.88,6.33,1.98,0.44,NaN
2,2024/04/12T11:33:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,83,52,63,1001.55,0.02,36616,...,22.04,70,70,2247.05,694.38,133.64,9.41,1.89,0.27,NaN
3,2024/04/12T11:35:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,83,51,63,1001.38,0.02,36552,...,20.67,68,68,2215.37,682.07,117.67,6.49,1.25,0.05,NaN
4,2024/04/12T11:37:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,84,51,64,1001.41,0.02,36552,...,17.63,62,62,2029.95,628.48,102.83,4.48,0.80,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144116,2024/12/17T15:23:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,97,49,75,1007.46,0.02,17440,...,44.20,123,108,4698.93,1454.25,276.49,21.09,1.05,0.02,NaN
144117,2024/12/17T15:25:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,97,49,75,1007.43,0.02,17440,...,44.96,121,107,4675.21,1452.37,257.02,20.89,3.96,1.02,NaN
144118,2024/12/17T15:27:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,97,50,75,1007.45,0.02,17440,...,41.60,112,102,4372.00,1347.27,234.85,19.42,4.23,0.70,NaN
144119,2024/12/17T15:29:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,96,50,74,1007.41,0.02,17440,...,44.98,122,108,4786.97,1487.24,261.38,20.59,3.79,1.00,NaN


In [81]:
df_tema_new = tema.set_index('UTCDateTime')

df_tema_new['tempC'] = Fahren_2_Celcius(df_tema_new['current_temp_f'])
df_tema_new.tempC

df_tema_new['pm25'] = correct_pm25(df_tema_new,'pm2_5_atm','tempC','current_humidity')
df_tema_new.columns

tema_new = df_tema_new[['pm2_5_atm','pm25','pm10_0_atm','tempC','current_humidity']]

tema_new['pm25'][tema_new['pm25'] < 0] = 0
tema_new

tema_new[['SO2','NOx','NH3','CO','O3']] = np.nan
tema_new = sub_index_calculation(tema_new)
tema_new = apply_aq_checks(tema_new)
tema_new= apply_aqi_calculation(tema_new)

tema_new['AQI']= tema_new['calculated_AQI'].apply(lambda x: categorize_AQI(x))

tema_new[['tempC','current_humidity','pm25','pm10_0_atm','calculated_AQI','AQI']]
#ksi_new.reset_index()
#df_ksi_new[['tempC','current_humidity','pm25','pm10_0_atm','calculated_AQI','AQI']]


,tempC,current_humidity,pm25,pm10_0_atm,calculated_AQI,AQI
UTCDateTime,,,,,,
2024/03/05T20:29:46z,22.0,19,0.0000,0.24,NaN,NaN
2024/04/12T11:31:44z,28.0,53,12.3136,20.20,21.0,Good
2024/04/12T11:33:44z,28.0,52,11.4106,19.61,20.0,Good
2024/04/12T11:35:44z,28.0,51,11.4526,18.91,19.0,Good
2024/04/12T11:37:44z,29.0,51,12.0970,17.13,20.0,Good
...,...,...,...,...,...,...
2024/12/17T15:23:48z,36.0,49,32.4490,39.66,54.0,Moderate
2024/12/17T15:25:48z,36.0,49,32.8756,42.19,55.0,Moderate
2024/12/17T15:27:48z,36.0,50,31.9534,39.39,53.0,Moderate


In [82]:
tema_final = tema_new[['tempC','current_humidity','pm25','pm10_0_atm','calculated_AQI','AQI']]

In [83]:
tema_final

,tempC,current_humidity,pm25,pm10_0_atm,calculated_AQI,AQI
UTCDateTime,,,,,,
2024/03/05T20:29:46z,22.0,19,0.0000,0.24,NaN,NaN
2024/04/12T11:31:44z,28.0,53,12.3136,20.20,21.0,Good
2024/04/12T11:33:44z,28.0,52,11.4106,19.61,20.0,Good
2024/04/12T11:35:44z,28.0,51,11.4526,18.91,19.0,Good
2024/04/12T11:37:44z,29.0,51,12.0970,17.13,20.0,Good
...,...,...,...,...,...,...
2024/12/17T15:23:48z,36.0,49,32.4490,39.66,54.0,Moderate
2024/12/17T15:25:48z,36.0,49,32.8756,42.19,55.0,Moderate
2024/12/17T15:27:48z,36.0,50,31.9534,39.39,53.0,Moderate


In [84]:
TEMA= tema_final.reset_index()
TEMA

,UTCDateTime,tempC,current_humidity,pm25,pm10_0_atm,calculated_AQI,AQI
0,2024/03/05T20:29:46z,22.0,19,0.0000,0.24,NaN,NaN
1,2024/04/12T11:31:44z,28.0,53,12.3136,20.20,21.0,Good
2,2024/04/12T11:33:44z,28.0,52,11.4106,19.61,20.0,Good
3,2024/04/12T11:35:44z,28.0,51,11.4526,18.91,19.0,Good
4,2024/04/12T11:37:44z,29.0,51,12.0970,17.13,20.0,Good
...,...,...,...,...,...,...,...
144116,2024/12/17T15:23:48z,36.0,49,32.4490,39.66,54.0,Moderate
144117,2024/12/17T15:25:48z,36.0,49,32.8756,42.19,55.0,Moderate
144118,2024/12/17T15:27:48z,36.0,50,31.9534,39.39,53.0,Moderate
144119,2024/12/17T15:29:48z,36.0,50,32.9740,42.55,55.0,Moderate


In [85]:
TEMA['UTCDateTime'] = TEMA['UTCDateTime'].str.replace("[A-Za-z]", " ")
TEMA['UTCDateTime'] = pd.to_datetime(TEMA['UTCDateTime'], 
                                      format='%Y/%m/%dT%H:%M:%Sz',
                                     exact=False)

In [86]:
TEMA.set_index('UTCDateTime').to_csv('tema_purple_air.csv')

In [87]:
pd.read_csv('tema_purple_air.csv')

,UTCDateTime,tempC,current_humidity,pm25,pm10_0_atm,calculated_AQI,AQI
0,2024-03-05 20:29:46,22.0,19,0.0000,0.24,NaN,NaN
1,2024-04-12 11:31:44,28.0,53,12.3136,20.20,21.0,Good
2,2024-04-12 11:33:44,28.0,52,11.4106,19.61,20.0,Good
3,2024-04-12 11:35:44,28.0,51,11.4526,18.91,19.0,Good
4,2024-04-12 11:37:44,29.0,51,12.0970,17.13,20.0,Good
...,...,...,...,...,...,...,...
144116,2024-12-17 15:23:48,36.0,49,32.4490,39.66,54.0,Moderate
144117,2024-12-17 15:25:48,36.0,49,32.8756,42.19,55.0,Moderate
144118,2024-12-17 15:27:48,36.0,50,31.9534,39.39,53.0,Moderate
144119,2024-12-17 15:29:48,36.0,50,32.9740,42.55,55.0,Moderate


In [88]:
pd.read_csv('kumasi_purple_air.csv')

,UTCDateTime,tempC,current_humidity,pm25,pm10_0_atm,calculated_AQI,AQI
0,2024-02-27 21:06:02,25.0,16,0.0000,0.00,NaN,NaN
1,2024-02-27 21:08:02,25.0,16,0.0000,0.00,NaN,NaN
2,2024-02-27 21:29:20,26.0,15,0.0000,0.00,NaN,NaN
3,2024-04-12 11:27:39,28.0,55,13.0180,22.20,22.0,Good
4,2024-04-12 11:29:39,28.0,54,12.2662,19.69,20.0,Good
...,...,...,...,...,...,...,...
73228,2024-08-26 15:25:36,31.0,61,30.7102,52.79,53.0,Moderate
73229,2024-08-26 15:27:36,31.0,61,31.3852,53.56,54.0,Moderate
73230,2024-08-26 15:29:36,31.0,62,46.9330,82.56,83.0,Moderate
73231,2024-08-26 15:31:36,31.0,62,34.9558,62.62,63.0,Moderate


In [89]:
pd.read_csv('old_fadama_purple_air.csv')

,UTCDateTime,tempC,current_humidity,pm25,pm10_0_atm,calculated_AQI,AQI
0,2024-03-05 20:29:39,22.0,19,0.0000,0.23,NaN,NaN
1,2024-04-12 11:30:17,28.0,51,11.1286,18.20,19.0,Good
2,2024-04-12 11:32:17,28.0,50,11.8888,19.57,20.0,Good
3,2024-04-12 11:34:17,28.0,50,11.8294,19.95,20.0,Good
4,2024-04-12 11:36:17,29.0,50,12.2578,18.27,20.0,Good
...,...,...,...,...,...,...,...
42306,2024-11-25 18:46:23,33.0,66,11.0800,0.00,NaN,NaN
42307,2024-11-25 18:48:23,33.0,66,11.0800,0.00,NaN,NaN
42308,2024-11-25 18:50:23,33.0,65,10.9600,0.00,NaN,NaN
42309,2024-11-25 18:52:23,33.0,65,10.9600,0.00,NaN,NaN


In [54]:
tema= pd.read_csv('stacked_tema.csv')
tema

,UTCDateTime,mac_address,firmware_ver,hardware,current_temp_f,current_humidity,current_dewpoint_f,pressure,adc,mem,...,pm10_0_atm_b,pm2.5_aqi_cf_1_b,pm2.5_aqi_atm_b,p_0_3_um_b,p_0_5_um_b,p_1_0_um_b,p_2_5_um_b,p_5_0_um_b,p_10_0_um_b,gas
0,2024/03/05T20:29:46z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,72,19,28,863.65,NaN,36816,...,0.00,0,0,71.61,21.24,3.67,0.35,0.00,0.0,NaN
1,2024/04/12T11:31:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,83,53,64,1001.48,NaN,38496,...,21.14,68,68,2207.44,686.84,117.88,6.33,1.98,0.44,NaN
2,2024/04/12T11:33:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,83,52,63,1001.55,0.02,36616,...,22.04,70,70,2247.05,694.38,133.64,9.41,1.89,0.27,NaN
3,2024/04/12T11:35:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,83,51,63,1001.38,0.02,36552,...,20.67,68,68,2215.37,682.07,117.67,6.49,1.25,0.05,NaN
4,2024/04/12T11:37:44z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,84,51,64,1001.41,0.02,36552,...,17.63,62,62,2029.95,628.48,102.83,4.48,0.80,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144116,2024/12/17T15:23:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,97,49,75,1007.46,0.02,17440,...,44.20,123,108,4698.93,1454.25,276.49,21.09,1.05,0.02,NaN
144117,2024/12/17T15:25:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,97,49,75,1007.43,0.02,17440,...,44.96,121,107,4675.21,1452.37,257.02,20.89,3.96,1.02,NaN
144118,2024/12/17T15:27:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,97,50,75,1007.45,0.02,17440,...,41.60,112,102,4372.00,1347.27,234.85,19.42,4.23,0.70,NaN
144119,2024/12/17T15:29:48z,d8:bc:38:84:48:8b,7.02,2.0+OPENLOG+31037 MB+DS3231+BME280+PMSX003-B+P...,96,50,74,1007.41,0.02,17440,...,44.98,122,108,4786.97,1487.24,261.38,20.59,3.79,1.00,NaN


In [55]:
print("Column Names:", tema.columns.tolist())

Column Names: ['UTCDateTime', 'mac_address', 'firmware_ver', 'hardware', 'current_temp_f', 'current_humidity', 'current_dewpoint_f', 'pressure', 'adc', 'mem', 'rssi', 'uptime', 'pm1_0_cf_1', 'pm2_5_cf_1', 'pm10_0_cf_1', 'pm1_0_atm', 'pm2_5_atm', 'pm10_0_atm', 'pm2.5_aqi_cf_1', 'pm2.5_aqi_atm', 'p_0_3_um', 'p_0_5_um', 'p_1_0_um', 'p_2_5_um', 'p_5_0_um', 'p_10_0_um', 'pm1_0_cf_1_b', 'pm2_5_cf_1_b', 'pm10_0_cf_1_b', 'pm1_0_atm_b', 'pm2_5_atm_b', 'pm10_0_atm_b', 'pm2.5_aqi_cf_1_b', 'pm2.5_aqi_atm_b', 'p_0_3_um_b', 'p_0_5_um_b', 'p_1_0_um_b', 'p_2_5_um_b', 'p_5_0_um_b', 'p_10_0_um_b', 'gas']


In [56]:
import pandas as pd
import os
import glob

# Set the folder path where your files are located
folder_path = "C:/Users/WINFRED VORDOAGU/Desktop/PYTHON/PD_DATA/KUMASI_PURPLEAIR"  # Change this to your actual folder path

# Get all CSV files in the folder
file_list = glob.glob(os.path.join(folder_path, "*.csv"))  # Change "*.csv" to "*.xlsx" for Excel files

# Initialize an empty list to store DataFrames
df_list = []

# Loop through the files and read them
for file in file_list:
    df = pd.read_csv(file)  # Use pd.read_excel(file) for Excel files
    df_list.append(df)

# Concatenate all DataFrames
stacked_df = pd.concat(df_list, ignore_index=True)

# Save the combined file
output_file = os.path.join(folder_path, "stacked_kumasi.csv")  # Change to .xlsx if needed
stacked_df.to_csv(output_file, index=False)  # Use stacked_df.to_excel(output_file, index=False) for Excel

print(f"Stacked file saved as: {output_file}")


Stacked file saved as: C:/Users/WINFRED VORDOAGU/Desktop/PYTHON/PD_DATA/KUMASI_PURPLEAIR\stacked_kumasi.csv
